<a href="https://colab.research.google.com/github/quynhu-d/TTS_FastSpeech/blob/master/Clean_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FastSpeech

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
# %cd ..
# !rm -r TTS_FastSpeech

/content


## Imports

In [1]:
!git clone https://github.com/quynhu-d/TTS_FastSpeech

Cloning into 'TTS_FastSpeech'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 129 (delta 52), reused 100 (delta 27), pack-reused 0
Receiving objects: 100% (129/129), 69.90 KiB | 6.35 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [2]:
%cd TTS_FastSpeech/

/content/TTS_FastSpeech


In [3]:
!pip install -r ./requirements.txt

     |████████████████████████████████| 1.7 MB 4.9 MB/s 
     |████████████████████████████████| 298 kB 56.6 MB/s 
     |████████████████████████████████| 140 kB 72.2 MB/s 
     |████████████████████████████████| 180 kB 57.8 MB/s 
     |████████████████████████████████| 97 kB 8.8 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 132 kB 59.6 MB/s 
     |████████████████████████████████| 243 kB 69.4 MB/s 
     |████████████████████████████████| 61 kB 647 kB/s 
     |████████████████████████████████| 1.1 MB 43.2 MB/s 
     |████████████████████████████████| 192 kB 67.9 MB/s 
     |████████████████████████████████| 160 kB 71.2 MB/s 
     |████████████████████████████████| 271 kB 70.3 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=3a817190a31fcf08b7d3f754b63460fa384070abf24db07c029bd280a29fb7a3
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb08

## Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from data import *

In [ ]:
# !wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
# !tar -xjf LJSpeech-1.1.tar.bz2 -C "/content/drive/MyDrive/TTS/"

--2021-12-05 11:35:28--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz 100%[===================>]   2.56G  16.3MB/s    in 2m 45s  

2021-12-05 11:38:14 (15.9 MB/s) - ‘LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



## Model
Параметры в основном такие же, как в статье.
FastSpeechConfig + MelSpectrogramConfig

In [6]:
# Fast Speech model configuration
from model import FastSpeechConfig
from featurizer import MelSpectrogramConfig
fconfig = FastSpeechConfig()
mel_config = MelSpectrogramConfig()
print(fconfig)
print(mel_config)

FastSpeechConfig(d_model=384, conv_hid_sz=1536, kernel_sz=3, d_k=384, d_v=384, dropout_rate=0.1, n_enc=6, n_dec=6, n_heads=2, max_phoneme_len=5000, max_mel_len=5000, n_mels=80)
MelSpectrogramConfig(sr=22050, win_length=1024, hop_length=256, n_fft=1024, f_min=0, f_max=8000, n_mels=80, power=1.0, pad_value=-11.5129251)


### Vocoder

In [7]:
!git clone https://github.com/NVIDIA/waveglow.git
!pip install googledrivedownloader

Cloning into 'waveglow'...
remote: Enumerating objects: 190, done.
remote: Total 190 (delta 0), reused 0 (delta 0), pack-reused 190
Receiving objects: 100% (190/190), 435.41 KiB | 7.26 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [8]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(
    file_id='1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF',
    dest_path='./waveglow_256channels_universal_v5.pt'
)

In [9]:
from model import Vocoder
%cd waveglow/
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vocoder = Vocoder()
%cd ..

/content/TTS_FastSpeech/waveglow


/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

/content/TTS_FastSpeech


## Overfitting

Оптимизатор -- Adam с параметрами для трансформеров (без планировщика)

In [ ]:
dataset = LJSpeechDataset('.')

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=3, collate_fn=LJSpeechCollator())

In [ ]:
iter(train_dataloader).next()

Batch(waveform=tensor([[-0.0007, -0.0008, -0.0006,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0003,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0025, -0.0037,  0.0005,  ...,  0.0003,  0.0002, -0.0004]]), waveform_length=tensor([212893,  41885, 213149], dtype=torch.int32), transcript=('Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition', 'in being comparatively modern.', 'For although the Chinese took impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands, by a similar process'), tokens=tensor([[27, 29, 20, 25, 31, 20, 25, 18,  6, 11, 20, 25, 11, 31, 19, 16, 11, 26,
         25, 23, 36, 11, 30, 16, 25, 30, 16, 11, 34, 20, 31, 19, 11, 34, 19, 20,
         14, 19, 11, 34, 16, 11, 12, 29, 16, 11, 12, 31, 11, 27, 29, 16, 30, 16,
         25, 31, 11, 14, 26, 25, 14, 16, 29, 25, 16, 15,  6, 11, 15, 20, 17, 17,
         16,

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import torch.nn.functional as F
from model.aligner import GraphemeAligner
from model.fastspeech import FastSpeech
import numpy as np
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FastSpeech(51, fconfig).to(device)
print(model)
aligner = GraphemeAligner().to(device)
optimizer = torch.optim.Adam(model.parameters(), 3e-4, (.9,.98))
n_epochs = 30000
wandb.init(name='overfit_batch_2')
featurizer = MelSpectrogram(MelSpectrogramConfig()).to(device)
for i in range(n_epochs):
    for batch in train_dataloader:
        batch.to(device)
        optimizer.zero_grad()
        batch.mel = featurizer(batch.waveform)
        durations = aligner(batch.waveform, batch.waveform_length, batch.transcript)
        durations *= batch.mel.size(-1)
        batch.durations = durations.round()
        batch.to(device)
        output = model(batch)
        batch.to(device)
        # print(batch.duration_preds.sum(1), batch.mel.size(-1), batch.durations.sum(1))
        dp_loss = F.mse_loss(batch.duration_preds, batch.durations)
        print(output.size(), batch.mel.size())
        sz_diff = np.abs(batch.mel.size(-1) - output.size(-1))
        if batch.mel.size(-1) > output.size(-1):
            print('padding batch')
            output = F.pad(output, (0, sz_diff, 0, 0, 0, 0), "constant", MelSpectrogramConfig.pad_value)
        else:
            print('padding mel')
            batch.mel = F.pad(batch.mel, (0, sz_diff, 0, 0, 0, 0), "constant", MelSpectrogramConfig.pad_value)        
        print(batch.mel.shape, output.shape)
        mel_loss = F.mse_loss(output, batch.mel)
        loss = mel_loss + dp_loss
        loss.backward()
        optimizer.step()
        wav = vocoder.inference(output.to(device))
        idx = np.random.randint(batch.mel.shape[0])
        wandb.log({
            'loss': loss,
            'mel_loss': mel_loss,
            'dp_loss': dp_loss,
            'mel': wandb.Image(batch.mel[idx]),
            'mel_pred': wandb.Image(output[idx]),
            'audio': wandb.Audio(batch.waveform[idx].detach().cpu().numpy(), sample_rate=MelSpectrogramConfig.sr),
            'audio_pred': wandb.Audio(wav[idx].detach().cpu().numpy(), sample_rate=MelSpectrogramConfig.sr),
            'step': i
        })
        break

FastSpeech(
  (emb_encoder): Embedding(51, 384)
  (phoneme_pe): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (phoneme_fft): Sequential(
    (0): FFTBlock(
      (pre_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (attention): MultiHeadAttention(
        (att_heads): ModuleList(
          (0): SelfAttention(
            (wq): Linear(in_features=384, out_features=384, bias=True)
            (wk): Linear(in_features=384, out_features=384, bias=True)
            (wv): Linear(in_features=384, out_features=384, bias=True)
          )
          (1): SelfAttention(
            (wq): Linear(in_features=384, out_features=384, bias=True)
            (wk): Linear(in_features=384, out_features=384, bias=True)
            (wv): Linear(in_features=384, out_features=384, bias=True)
          )
        )
        (w_out): Linear(in_features=768, out_features=384, bias=True)
      )
      (att_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (co

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth


  0%|          | 0.00/360M [00:00<?, ?B/s]

wandb: Currently logged in as: quynhu_d (use `wandb login --relogin` to force relogin)


torch.Size([3, 80, 800]) torch.Size([3, 80, 833])
padding batch
torch.Size([3, 80, 833]) torch.Size([3, 80, 833])
torch.Size([3, 80, 800]) torch.Size([3, 80, 833])
padding batch
torch.Size([3, 80, 833]) torch.Size([3, 80, 833])
torch.Size([3, 80, 800]) torch.Size([3, 80, 833])
padding batch
torch.Size([3, 80, 833]) torch.Size([3, 80, 833])
torch.Size([3, 80, 800]) torch.Size([3, 80, 833])
padding batch
torch.Size([3, 80, 833]) torch.Size([3, 80, 833])
torch.Size([3, 80, 800]) torch.Size([3, 80, 833])
padding batch
torch.Size([3, 80, 833]) torch.Size([3, 80, 833])
torch.Size([3, 80, 800]) torch.Size([3, 80, 833])
padding batch
torch.Size([3, 80, 833]) torch.Size([3, 80, 833])
torch.Size([3, 80, 800]) torch.Size([3, 80, 833])
padding batch
torch.Size([3, 80, 833]) torch.Size([3, 80, 833])
torch.Size([3, 80, 800]) torch.Size([3, 80, 833])
padding batch
torch.Size([3, 80, 833]) torch.Size([3, 80, 833])
torch.Size([3, 80, 800]) torch.Size([3, 80, 833])
padding batch
torch.Size([3, 80, 833])

## Training

In [ ]:
from trainer import train, TrainConfig
train_config = TrainConfig()
train_config.save_dir = '/content/drive/MyDrive/TTS/saved/'
train_config.wandb_name = 'run_1'
train_config.lj_path = '/content/drive/MyDrive/TTS/'
train_config.batch_size = 3
train_config.display_every = 100
print(train_config)

TrainConfig(wandb_project='TTS_FastSpeech', wandb_name='run_1', display_every=100, lj_path='/content/drive/MyDrive/TTS/', batch_size=3, val_split=None, n_epochs=50, lr=0.0003, save_dir='/content/drive/MyDrive/TTS/saved/')


Saving model at /content/drive/MyDrive/TTS/saved//05-12-02-22-34/

In [ ]:
train(train_config, mel_config, fconfig, vocoder)

Training on cuda
Saving model at /content/drive/MyDrive/TTS/saved//05-12-02-22-34/


Streaming output truncated to the last 5000 lines.
('on the other, the governor of the jail sneered at his zeal', 'stigmatized his often most just strictures on abuses as "a bundle of nonsense," and the aldermen, when he appealed to them for protection and countenance,', 'generally sided with his opponents. Nevertheless the inspectors summed up against him.')
Predicted duration length:152, aligner length: 154
Batch skipped
('While admitting that he had had many difficulties to contend with,', 'and that he had again and again protested against the obstacles thrown in his way, the inspectors "cannot forbear expressing their opinion that he might have shown greater perseverance,', 'in the face of impediments confessedly discouraging')
Predicted duration length:184, aligner length: 185
Batch skipped
('as regards the private teaching of prisoners; and they went on to say that "a resolved adherence, in spite of discouragements the most disheartening,', 'to that line of conduct which his duty

## Inference

In [10]:
from tester import test

In [14]:
transcript = [
    "A defibrillator is a device that gives a high energy electric shock to the heart of someone who is in cardiac arrest",
    "Massachusetts Institute of Technology may be best known for its math, science and engineering education",
    "Wasserstein distance or Kantorovich Rubinstein metric is a distance function defined between probability distributions on a given metric space"
]
model_path = '/content/drive/MyDrive/TTS/saved/06-12-08-28-08/best_loss_model.pth'
# for fconfig and vocoder see above
reconstructed_wavs = test(vocoder.to('cuda'), fconfig, model_path, transcript)

(лучше не слушать))

In [15]:
from IPython import display
for wav in reconstructed_wavs:
    display.display(display.Audio(wav.detach().cpu().numpy(), rate=22050))